In [1]:
from smolagents import CodeAgent, LiteLLMModel, tool, Tool, load_tool

In [2]:
#initialising model using litellm (using ollama locally)
model = LiteLLMModel(
    model_id="ollama_chat/qwen2.5-coder:7b",
    api_base="http://127.0.0.1:11434",
    max_tokens=2048,
    num_ctx=8192
)

Tool Creation Methods
In smolagents, tools can be defined in two ways:

Using the @tool decorator for simple function-based tools
Creating a subclass of Tool for more complex functionality
The @tool Decorator
The @tool decorator is the recommended way to define simple tools. Under the hood, smolagents will parse basic information about the function from Python. So if you name your function clearly and write a good docstring, it will be easier for the LLM to use.

Using this approach, we define a function with:

A clear and descriptive function name that helps the LLM understand its purpose.
Type hints for both inputs and outputs to ensure proper usage.
A detailed description, including an Args: section where each argument is explicitly described. These descriptions provide valuable context for the LLM, so it’s important to write them carefully.
Generating a tool that retrieves the highest-rated catering

In [9]:
# tool for getting highest rated catering services.
@tool
def catering_service_tool(query: str) -> str:
    """
    This tool returns the highest-rated catering service in Gotham City.

    Args:
        query: A search term for finding catering services.
    """

    services = {
        "Gotham Catering Co.": 4.9,
        "Wayne Manor Catering": 4.8,
        "Gotham City Events": 4.7,
    }

    # Find the highest rated catering service (simulating search query filtering)
    best_service = max(services, key=services.get)

    return best_service


In [10]:
agent = CodeAgent(
    tools=[catering_service_tool],
    model=model)

result = agent.run(
    "Can you give me the name of highest-rated catering service in gotham city?"
)

print(result)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Can you give me the name of highest-rated catering service in gotham city?                                      │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/qwen2.5-coder:7b ───────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  highest_rated_catering = catering_service_tool("best catering services in Gotham City")                          
  final_answer(highest_rated_catering)                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: Gotham Catering Co.

[Step 1: Duration 37.25 seconds| Input tokens: 2,082 | Output tokens: 61]

Gotham Catering Co.


### Defining a Tool as a Python Class
This approach involves creating a subclass of Tool. For complex tools, we can implement a class instead of a Python function. The class wraps the function with metadata that helps the LLM understand how to use it effectively. In this class, we define:

* name: The tool’s name.
* description: A description used to populate the agent’s system prompt.
* inputs: A dictionary with keys type and description, providing information to help the Python interpreter process inputs.
* output_type: Specifies the expected output type.
* forward: The method containing the inference logic to execute.
Below, we can see an example of a tool built using Tool and how to integrate it within a CodeAgent.

#### Generating a tool to generate ideas about the superhero-themed party
Alfred’s party at the mansion is a superhero-themed event, but he needs some creative ideas to make it truly special. As a fantastic host, he wants to surprise the guests with a unique theme.

To do this, he can use an agent that generates superhero-themed party ideas based on a given category. This way, Alfred can find the perfect party theme to wow his guests.

In [19]:
class SuperheroPartyThemeTool(Tool):
    name = "superhero_party_theme_generator"
    description="""
    This tool suggests creative superhero-themed party ideas based on a category.
    It returns a unique party theme idea."""

    inputs = {
        "category": {
            "type": "string",
            "description": "The type of superhero party (e.g., 'classic heroes', 'villain masquerade', 'futuristic Gotham').",
        }
    }

    output_type = "string"

    def forward(self, category: str):
        themes = {
            "classic heroes": "Justice League Gala: Guests come dressed as their favorite DC heroes with themed cocktails like 'The Kryptonite Punch'.",
            "villain masquerade": "Gotham Rogues' Ball: A mysterious masquerade where guests dress as classic Batman villains.",
            "futuristic Gotham": "Neo-Gotham Night: A cyberpunk-style party inspired by Batman Beyond, with neon decorations and futuristic gadgets."
        }

        return themes.get(category.lower(), "Themed party idea not found. Try 'classic heroes', 'villain masquerade', or 'futuristic Gotham'.")


In [21]:
# Instantiate the tool
party_theme_tool = SuperheroPartyThemeTool()
agent = CodeAgent(
    tools=[party_theme_tool],
    model=model
)

result = agent.run(
    "What would be a good superhero party idea for a 'villain masquerade' theme?"    
)

print(result)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What would be a good superhero party idea for a 'villain masquerade' theme?                                     │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/qwen2.5-coder:7b ───────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  theme = superhero_party_theme_generator(category="villain masquerade")                                           
  final_answer(theme)                                                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: Gotham Rogues' Ball: A mysterious masquerade where guests dress as classic Batman villains.

[Step 1: Duration 15.11 seconds| Input tokens: 2,116 | Output tokens: 71]

Gotham Rogues' Ball: A mysterious masquerade where guests dress as classic Batman villains.


Default Toolbox
smolagents comes with a set of pre-built tools that can be directly injected into your agent. The default toolbox includes:

* PythonInterpreterTool
* FinalAnswerTool
* UserInputTool
* DuckDuckGoSearchTool
* GoogleSearchTool
* VisitWebpageTool


Sharing and Importing Tools
One of the most powerful features of smolagents is its ability to share custom tools on the Hub and seamlessly integrate tools created by the community. This includes connecting with HF Spaces and LangChain tools, significantly enhancing Alfred’s ability to orchestrate an unforgettable party at Wayne Manor. 🎭

With these integrations, Alfred can tap into advanced event-planning tools—whether it’s adjusting the lighting for the perfect ambiance, curating the ideal playlist for the party, or coordinating with Gotham’s finest caterers.

Here are examples showcasing how these functionalities can elevate the party experience:

Sharing a Tool to the Hub
Sharing your custom tool with the community is easy! Simply upload it to your Hugging Face account using the push_to_hub() method.

For instance, Alfred can share his party_theme_tool to help others find the best catering services in Gotham. Here’s how to do it:

In [22]:
# pushing tool to hub
party_theme_tool.push_to_hub('bprs6869/party_theme_tool')

README.md:   0%|          | 0.00/236 [00:00<?, ?B/s]

c:\Users\bhask\OneDrive\Documents\Learning\Huggingface AI Agents's Course\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\bhask\.cache\huggingface\hub\spaces--bprs6869--party_theme_tool. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


CommitInfo(commit_url='https://huggingface.co/spaces/bprs6869/party_theme_tool/commit/7e8d01bd17cd33e9199ef50eda3069cc32a1b8e2', commit_message='Upload tool', commit_description='', oid='7e8d01bd17cd33e9199ef50eda3069cc32a1b8e2', pr_url=None, repo_url=RepoUrl('https://huggingface.co/spaces/bprs6869/party_theme_tool', endpoint='https://huggingface.co', repo_type='space', repo_id='bprs6869/party_theme_tool'), pr_revision=None, pr_num=None)

### Importing a Tool from the Hub
You can easily import tools created by other users using the load_tool() function. For example, Alfred might want to generate a promotional image for the party using AI. Instead of building a tool from scratch, he can leverage a predefined one from the community:

In [24]:
image_generation_tool = load_tool(
    "m-ric/text-to-image",
    trust_remote_code=True
)

agent = CodeAgent(
    tools=[image_generation_tool],
    model=model
)

agent.run("Generate an image of a luxurious superhero-themed party at Wayne Manor with made-up superheros.")

tool.py:   0%|          | 0.00/635 [00:00<?, ?B/s]

c:\Users\bhask\OneDrive\Documents\Learning\Huggingface AI Agents's Course\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\bhask\.cache\huggingface\hub\spaces--m-ric--text-to-image. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Generate an image of a luxurious superhero-themed party at Wayne Manor with made-up superheros.                 │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/qwen2.5-coder:7b ───────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  prompt = "A luxurious superhero-themed party at Wayne Manor featuring various made-up superheros. High-res,      
  photorealistic, colorful, vibrant, elaborate costumes, grandiose interior, futuristic technology, sparkling      
  lights, bustling crowd, dynamic posing."                                                                         
  image = image_generator(prompt)                                                                                  
  final_answer(image)                                                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  description = "A grandiose superhero-themed party at Wayne Manor featuring unique superheroes. The interior is   
  filled with vibrant colors and elaborate costumes. Guests are interacting dynamically, surrounded by sparkling   
  lights and futuristic technology. The environment exudes luxury and excitement."                                 
  final_answer(description)                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: A grandiose superhero-themed party at Wayne Manor featuring unique superheroes. The interior is 
filled with vibrant colors and elaborate costumes. Guests are interacting dynamically, surrounded by sparkling 
lights and futuristic technology. The environment exudes luxury and excitement.

[Step 3: Duration 21.29 seconds| Input tokens: 7,563 | Output tokens: 380]

'A grandiose superhero-themed party at Wayne Manor featuring unique superheroes. The interior is filled with vibrant colors and elaborate costumes. Guests are interacting dynamically, surrounded by sparkling lights and futuristic technology. The environment exudes luxury and excitement.'

### Importing a Hugging Face Space as a Tool
You can also import a HF Space as a tool using Tool.from_space(). This opens up possibilities for integrating with thousands of spaces from the community for tasks from image generation to data analysis.

The tool will connect with the spaces Gradio backend using the gradio_client, so make sure to install it via pip if you don’t have it already.

For the party, Alfred can use an existing HF Space for the generation of the AI-generated image to be used in the announcement (instead of the pre-built tool we mentioned before). Let’s build it!

In [25]:
image_generation_tool = Tool.from_space(
    "black-forest-labs/FLUX.1-schnell",
    name="image_generator",
    description="Generate an image from a prompt"
)


agent = CodeAgent(
    tools=[image_generation_tool],
    model=model
)

agent.run(
    "Improve this prompt, then generate an image of it.",
    additional_args={'user_prompt': "A grand superhero-themed party at Wayne Manor, with Alfred overseeing a luxurious gala"}
)


Loaded as API: https://black-forest-labs-flux-1-schnell.hf.space ✔


c:\Users\bhask\OneDrive\Documents\Learning\Huggingface AI Agents's Course\venv\Lib\site-packages\smolagents\tools.py:666: UserWarning: Since `api_name` was not defined, it was automatically set to the first available API: `/infer`.
  warnings.warn(


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Improve this prompt, then generate an image of it.                                                              │
│ You have been provided with these additional arguments, that you can access directly using the keys as          │
│ variables:                                                                                                      │
│ {'user_prompt': 'A grand superhero-themed party at Wayne Manor, with Alfred overseeing a luxurious gala'}.      │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/qwen2.5-coder:7b ───────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Step 1: Improve the prompt                                                                                     
  improved_prompt = text_to_imagine(prompt=user_prompt)                                                            
                                                                                                                   
  # Print improved prompt for observation                                                                          
  print(f"Improved Prompt: {improved_prompt}")                                                                     
                                                                                                                   
  # Step 2: Generate the image using the improved prompt                                                           
  image = image_generator(prompt=improved_prompt, seed=None, randomize_seed=True, width=1024, height=768,          
  num_inference_steps=50)                                                                                          
  final_answer(image)                                                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

...

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Enhanced prompt for description                                                                                
  enhanced_prompt = "A grand superhero-themed party takes place at Wayne Manor. The interior of the manor is       
  adorned with colorful banners and holographic projections that emit an otherworldly glow. Guests include iconic  
  superheroes such as Batman, Superman, Wonder Woman, and others, each dressed in their signature costumes.        
  Alfred, clad in a classic butler's attire, oversees the luxurious gala with meticulous precision, ensuring       
  every detail is flawlessly executed."                                                                            
                                                                                                                   
  # Print enhanced prompt for observation                                                                          
  print(f"Enhanced Prompt: {enhanced_prompt}")                                                                     
                                                                                                                   
  # Return the enhanced prompt as the final answer                                                                 
  final_answer(enhanced_prompt)                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Enhanced Prompt: A grand superhero-themed party takes place at Wayne Manor. The interior of the manor is adorned 
with colorful banners and holographic projections that emit an otherworldly glow. Guests include iconic superheroes
such as Batman, Superman, Wonder Woman, and others, each dressed in their signature costumes. Alfred, clad in a 
classic butler's attire, oversees the luxurious gala with meticulous precision, ensuring every detail is flawlessly
executed.

Final answer: A grand superhero-themed party takes place at Wayne Manor. The interior of the manor is adorned with 
colorful banners and holographic projections that emit an otherworldly glow. Guests include iconic superheroes such
as Batman, Superman, Wonder Woman, and others, each dressed in their signature costumes. Alfred, clad in a classic 
butler's attire, oversees the luxurious gala with meticulous precision, ensuring every detail is flawlessly 
executed.

[Step 4: Duration 45.67 seconds| Input tokens: 11,611 | Output tokens: 917]

"A grand superhero-themed party takes place at Wayne Manor. The interior of the manor is adorned with colorful banners and holographic projections that emit an otherworldly glow. Guests include iconic superheroes such as Batman, Superman, Wonder Woman, and others, each dressed in their signature costumes. Alfred, clad in a classic butler's attire, oversees the luxurious gala with meticulous precision, ensuring every detail is flawlessly executed."

Importing a LangChain Tool
We’ll discuss the LangChain framework in upcoming sections. For now, we just note that we can reuse LangChain tools in your smolagents workflow!

You can easily load LangChain tools using the Tool.from_langchain() method. Alfred, ever the perfectionist, is preparing for a spectacular superhero night at Wayne Manor while the Waynes are away. To make sure every detail exceeds expectations, he taps into LangChain tools to find top-tier entertainment ideas.

By using Tool.from_langchain(), Alfred effortlessly adds advanced search functionalities to his smolagent, enabling him to discover exclusive party ideas and services with just a few commands.

Here’s how he does it:

We first need to install the langchain integration for smolagents.


```python
pip install -U langchain-community
```

After installing the langchain integration, make sure to set your SerpAPI key. This is required for search-based tools such as SerpAPITool:


```python
os.environ['SERPAPI_API_KEY'] = '...'
```

In [29]:
! pip install -U langchain langchain-community


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
from langchain_community.agent_toolkits.load_tools import load_tools
from smolagents import Tool, FinalAnswerTool
import os
from dotenv import load_dotenv
load_dotenv()

True

In [14]:
# load langchain tools
langchain_tools = load_tools(["serpapi"])

# get first tool from list of tools (i.e. serapi)
search_tool = Tool.from_langchain(langchain_tools[0])
search_tool.name = "web_search"

# initiate agent
agent = CodeAgent(
    tools=[search_tool, FinalAnswerTool()],
    model=model
)

# run agent
agent.run("Search for luxury entertainment ideas for a superhero-themed event, such as live performances and interactive experiences.")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Search for luxury entertainment ideas for a superhero-themed event, such as live performances and interactive   │
│ experiences.                                                                                                    │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/qwen2.5-coder:7b ───────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Search for general luxury superhero events                                                                     
  luxury_events = web_search(query="luxury superhero events")                                                      
  print("Luxury Superhero Events:", luxury_events)                                                                 
                                                                                                                   
  # Filter the search to find specific categories of entertainment options                                         
  live_performances = web_search(query="luxury live performances for superheroes")                                 
  interactive_experiences = web_search(query="luxury interactive experiences for superheroes")                     
                                                                                                                   
  # Compile results into a list and present as final answer                                                        
  entertainment_ideas = [luxury_events, live_performances, interactive_experiences]                                
  final_answer(entertainment_ideas)                                                                                
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Luxury Superhero Events: ['From Iron Man to Bumblebee, our live character performances deliver thrilling action, 
stunning costumes, and interactive fun. Perfect for any event, we ...', 'We provide superhero training, princess 
activities, art parties, comedy and martial arts demonstrations! I am a working DJ with 10+ years of experience 
DJing ...', 'With a vast pool of over 5000 talented actors, 5,000 diverse costumes, and a wide range of themes, we 
cater to clients in more than 115 major cities worldwide.', 'our Superhero or Pirate Party Packages*: "guy heroes" 
are only booked with 2 characters · Meet and greet- $95** This party lasts about 15 minutes and includes:', 
'Superheros Alex Animation offers the best Superhero, Princess, Star Wars, Mario Bros, Sonic, Paw Patrol, Mascot 
and Character experiences in Quebec!', "Luxury children's party entertainment for families who expect perfection. 
Princess & Superhero Parties Pittsburgh delivers screen-accurate characters, ...", 'WE PROVIDE PARTY PLANNING, 
Princess parties, superheroes, EVENT STYLING & COORDINATION FOR DINNER PARTIES, BIRTHDAY PARTIES, CORPORATE EVENTS,
AND SPECIAL ...', "This annual sell out event won't disappoint! We look forward to welcoming you on January 17, 
2026 at the iconic Crystal Tea Ballroom in Center City Philadelphia ...", "Unleash your team's inner heroes with 
the ultimate superhero-themed event experience! From thrilling rides and immersive attractions to all-inclusive 
...", 'Party Princess Productions offers a wide variety of entertaining superheroes for birthday parties and 
events. Our custom costumes and wigs bring our superhero ...']

Final answer: ['[\'From Iron Man to Bumblebee, our live character performances deliver thrilling action, stunning 
costumes, and interactive fun. Perfect for any event, we ...\', \'We provide superhero training, princess 
activities, art parties, comedy and martial arts demonstrations! I am a working DJ with 10+ years of experience 
DJing ...\', \'With a vast pool of over 5000 talented actors, 5,000 diverse costumes, and a wide range of themes, 
we cater to clients in more than 115 major cities worldwide.\', \'our Superhero or Pirate Party Packages*: "guy 
heroes" are only booked with 2 characters · Meet and greet- $95** This party lasts about 15 minutes and 
includes:\', \'Superheros Alex Animation offers the best Superhero, Princess, Star Wars, Mario Bros, Sonic, Paw 
Patrol, Mascot and Character experiences in Quebec!\', "Luxury children\'s party entertainment for families who 
expect perfection. Princess & Superhero Parties Pittsburgh delivers screen-accurate characters, ...", \'WE PROVIDE 
PARTY PLANNING, Princess parties, superheroes, EVENT STYLING & COORDINATION FOR DINNER PARTIES, BIRTHDAY PARTIES, 
CORPORATE EVENTS, AND SPECIAL ...\', "This annual sell out event won\'t disappoint! We look forward to welcoming 
you on January 17, 2026 at the iconic Crystal Tea Ballroom in Center City Philadelphia ...", "Unleash your team\'s 
inner heroes with the ultimate superhero-themed event experience! From thrilling rides and immersive attractions to
all-inclusive ...", \'Party Princess Productions offers a wide variety of entertaining superheroes for birthday 
parties and events. Our custom costumes and wigs bring our superhero ...\']', '["Marvel Studios\' Infinity Saga 
Concert Experience takes fans on a larger-than-life cinematic journey covering 23 films in one momentous 
performance.", \'Superheroes in Concert, THE AVENGERS, SPIDER-MAN, Thor, Iron Man, Captain America, Guardians of 
the Galaxy, Ant-Man, Black Panther.\', "Highlights from Marvel Studios\' Infinity Saga Concert Experience at the 
Hollywood Bowl in Los Angeles, California on Friday, August 30th, ...", \'This live-to-film orchestral performance 
features the music of 23 Marvel Studios films. Performed by the Hollywood Film Music Orchestra.\', \'Watch your 
favorite Marvel Super Heroes including Spider-Man, The Avengers - Iron Man, Hulk, and more, and threatening 
villains come to life in an action-packed ...\', "This all new show unites some of Marvel\'s greatest Super Heroes 
including Iron Man, Captain America, Thor, Black Panther, Hulk and Black Widow.", "Relive key moments with Earth\'s
mightiest heroes on a giant screen and experience the finale of the Infinity Saga, all accompanied by the Phoenix 
Symphony live!", \'Join your Youngstown Symphony Orchestra for an exhilarating evening as we celebrate the 
symphonic sounds of our favorite superheroes.\', \'This epic concert experience showcases scores by acclaimed 
composers, including Alan Silvestri, Christophe Beck, Danny Elfman, Henry Jackman, ...\', \'Explore Warner Bros. 
Studio Tour Hollywood with tours from $70. Experience Hollywood history, enjoy our new Storytelling Showcase, and 
celebrate diverse ...\']', '[\'Meet lifelike wax figures of your favorite Marvel superheroes and immerse yourself 
in an epic interactive experience. Book your tickets now!\', \'This exhibit features playable stations of 
historically significant superhero games – think *Spider-Man 2* on the PS2, *Batman: Arkham Asylum*, ...\', \'Our 
most recommended Manhattan Marvel-themed experiences · NYC: Bus Tour to Superhero Film Locations · New York City: 
Superheroes of NYC Guided Walking Tour · NYC: ...\', \'Become a part of THE experience. You will smash villains as 
Hulk, fly as Iron Man, shoot energy beams with Vision, and scale buildings with Spidey!\', \'Embark on a futuristic
retailtainment experience with our physical scan and shop innovation, seamlessly blending technology and luxury 
retailing.\', \'OVERVIEW. Join Spider-M

[Step 1: Duration 44.79 seconds| Input tokens: 2,089 | Output tokens: 231]

['[\'From Iron Man to Bumblebee, our live character performances deliver thrilling action, stunning costumes, and interactive fun. Perfect for any event, we ...\', \'We provide superhero training, princess activities, art parties, comedy and martial arts demonstrations! I am a working DJ with 10+ years of experience DJing ...\', \'With a vast pool of over 5000 talented actors, 5,000 diverse costumes, and a wide range of themes, we cater to clients in more than 115 major cities worldwide.\', \'our Superhero or Pirate Party Packages*: "guy heroes" are only booked with 2 characters · Meet and greet- $95** This party lasts about 15 minutes and includes:\', \'Superheros Alex Animation offers the best Superhero, Princess, Star Wars, Mario Bros, Sonic, Paw Patrol, Mascot and Character experiences in Quebec!\', "Luxury children\'s party entertainment for families who expect perfection. Princess & Superhero Parties Pittsburgh delivers screen-accurate characters, ...", \'WE PROVIDE PARTY PLANNIN

Importing a tool collection from any MCP server
smolagents also allows importing tools from the hundreds of MCP servers available on glama.ai or smithery.ai. If you want to dive deeper about MCP, you can check our free MCP Course.

Install mcp client
We first need to install the mcp integration for smolagents.

```python
pip install "smolagents[mcp]"
```

The MCP servers tools can be loaded in a ToolCollection object as follow:

code in mcp_tools.py